In [1]:
# importing libraries

import open3d, ctypes, os
from glob import glob

# Data augmentation

In [4]:
from data_augmentation import data_preprocessing

# read all the files in the folder dataset/original_dataset
files = glob("./new_dataset/train/input/quads/*")

for file in files:
    data_preprocessing(file)


Preprocessing ./new_dataset/train/input/quads/female_0279.obj ...
Preprocessing ./new_dataset/train/input/quads/karate_0239.obj ...
Preprocessing ./new_dataset/train/input/quads/karate_0205.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0245.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0523.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0537.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0251.obj ...
Preprocessing ./new_dataset/train/input/quads/karate_0211.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0286.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0292.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0047.obj ...
Preprocessing ./new_dataset/train/input/quads/karate_0007.obj ...
Preprocessing ./new_dataset/train/input/quads/karate_0013.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0053.obj ...
Preprocessing ./new_dataset/train/input/quads/female_0084.obj ...
Preprocess

In [2]:
from generation_orientation_field import generate_output
import numpy as np

files_quads = glob("./new_dataset/train/input/quads/*")

for file in files_quads:
  file_name = os.path.basename(file)
  file_triangle = "./new_dataset/train/input/triangles/" + file_name

  orientation_fields = generate_output(file_triangle, file)
  orientation_fields_reshaped = orientation_fields.reshape(orientation_fields.shape[0], -1)


  # save the orientation fields in a txt file
  output_file = "./new_dataset/train/output/" + file_name
  np.savetxt(output_file, orientation_fields_reshaped)


  print("Orientation fields saved in: ", output_file)

Orientation fields saved in:  ./new_dataset/train/output/female_0279.obj
Orientation fields saved in:  ./new_dataset/train/output/karate_0239.obj
Orientation fields saved in:  ./new_dataset/train/output/karate_0205.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0245.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0523.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0537.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0251.obj
Orientation fields saved in:  ./new_dataset/train/output/karate_0211.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0286.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0292.obj
Orientation fields saved in:  ./new_dataset/train/output/female_0047.obj
Orientation fields saved in:  ./new_dataset/train/output/karate_0007.obj
Orientation fields saved in:  ./new_dataset/train/output/karate_0013.obj
Orientation fields saved in:  ./new_dataset/train/o

## Loading mesh to retopologize

In [7]:
file_path = "./male.obj" # change this URL to the path of the file (male.obj is a test file)

'''
# open the file
with open(file_path, "r+") as file:
  lines = file.readlines()
  file.seek(0)  # Posiziona il cursore all'inizio del file

  # Scrivi solo le linee che iniziano con 'v' o 'f'
  for line in lines:
      if (line[0] == "v" or line[0] == "f") and line[1] == " ":
          file.write(line)

  # Tronca il file alla lunghezza attuale per rimuovere il contenuto rimanente
  file.truncate()

file.close()
'''

mesh = open3d.io.read_triangle_mesh(file_path)

vertices = mesh.vertices
triangles = mesh.triangles

num_vertices = len(vertices)
num_triangles = len(triangles)

# create file input.txt
file = open("input.txt", "w")

# write vertices
for i in range(len(vertices)):
    file.write(str(vertices[i][0]) + " " + str(vertices[i][1]) + " " + str(vertices[i][2]) + "\n")

# write triangles
for i in range(len(triangles)):
    file.write(str(triangles[i][0]) + " " + str(triangles[i][1]) + " " + str(triangles[i][2]) + "\n")


file.close()

## Detection of Crest Lines

In [8]:
# loading the shared library with C++
shared_library_path = glob(os.path.join("./build", "**/*.so"), recursive=True)
assert shared_library_path, "No shared library found, please build the shared library first (README.md for instructions)"
shared_library = ctypes.CDLL(shared_library_path[0])

k = 3 # neighborhood size

# detect crest lines
shared_library.crest_lines(num_vertices, num_triangles, k)

1

## View crest lines generated

In [ ]:
from viewer import mappa_vertici

